## Dependencies
- [goatools](https://github.com/tanghaibao/goatools)
- [go-basic.obo](http://geneontology.org/ontology/go-basic.obo)
- [Human GO annotation file](http://current.geneontology.org/annotations/goa_human.gaf.gz)

In [ ]:
import numpy as np
import pandas as pd 
import umap
import matplotlib.pyplot as plt
from Bio import SeqIO
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

In [ ]:
client_set=set(SeqIO.index("../fig1/result/drllps_client_clstr_Homo_sapiens.fasta", "fasta").keys())
scaffold_set=set(SeqIO.index("../fig1/result/drllps_scaffold_clstr_Homo_sapiens.fasta", "fasta").keys())
nonllps_set=set(SeqIO.index("../fig1/result/drllps_nonllps_clstr_Homo_sapiens.fasta", "fasta").keys())
    
mat=np.load("../fig2/PTT5XLU50_human.npy", allow_pickle=True)
mat=mat.item()

list_client=[]
list_nonllps=[]
list_scaffold=[]
client_id=[]
scaffold_id=[]
nonllps_id=[]
for k in mat.keys():
    if k in nonllps_set:
        list_nonllps.append(mat[k])
        nonllps_id.append(k)
    elif k in client_set:
        list_client.append(mat[k])
        client_id.append(k)
    elif k in scaffold_set:
        list_scaffold.append(mat[k])
        scaffold_id.append(k)

In [ ]:
x = np.array(list_client + list_nonllps)
y = np.array([True]*len(list_client) + [False]*len(list_nonllps))
ids = np.array(client_id + nonllps_id)

In [ ]:
cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
y_tests=[]
y_preds=[]
y_fp=[]
for train,test in cv.split(x, y):
    model=make_pipeline(StandardScaler(), SVC(class_weight="balanced", probability=True))
    model.fit(x[train],y[train])
    y_pred_p=model.predict_proba(x[test])[:,1]
    y_pred=model.predict(x[test])
    y_tests.append(y[test])
    y_preds.append(y_pred)
    y_pred_f=y_pred[y[test]==False]
    y_fp.append(ids[test][y[test]==False][y_pred_f==True])

In [ ]:
with open("data/fp_id.txt", "w") as f:
    for cv in y_fp:
        for i in cv:
            uni=i.split("|")[1]
            f.write(uni+"\n")

In [ ]:
with open("../result/fp_id_fullid.txt", "w") as f:
    for cv in y_fp:
        for i in cv:
            f.write(i+"\n")

In [ ]:
with open("data/human_client.txt", "w") as f:
    for name in client_set:
        uni=name.split("|")[1]
        f.write(uni+"\n")
with open("data/human_nonllps.txt", "w") as f:
    for name in nonllps_set:
        uni=name.split("|")[1]
        f.write(uni+"\n")
with open("data/human_client_nonllps.txt", "w") as f:
    for name in nonllps_set:
        uni=name.split("|")[1]
        f.write(uni+"\n")
    for name in client_set:
        uni=name.split("|")[1]
        f.write(uni+"\n")

In [ ]:
!find_enrichment.py data/fp_id.txt data/human_nonllps.txt data/goa_human.gaf --pval=-1 --method=fdr_bh --pval_field=fdr_bh --outfile=result/fp_cc_go.txt --ns=CC
!find_enrichment.py data/human_client.txt data/human_client_nonllps.txt data/goa_human.gaf --pval=-1 --method=fdr_bh --pval_field=fdr_bh --outfile=result/client_go.txt --ns=CC